In [1]:
import sys
if '/Users/nehiljain/code/find-your-mate-ai/src' not in sys.path:
    sys.path.append('/Users/nehiljain/code/find-your-mate-ai/src')

import nest_asyncio

nest_asyncio.apply()
from find_your_mate_ai.config import settings
from find_your_mate_ai.data_ingestion import *
import pandas as pd
documents = ingest_profiles_data(
        source_data_path=settings.SOURCE_DATA_PATH,
        output_data_path=settings.OUTPUT_DATA_PATH,
    )
documents_df = pd.DataFrame([doc.to_dict() for doc in documents])
metadata_df = documents_df['metadata'].apply(pd.Series)
documents_df = pd.concat([documents_df.drop(columns=['metadata']), metadata_df], axis=1)
documents_df.shape

INFO:root:Starting data ingestion from /Users/nehiljain/code/find-your-mate-ai/data
Starting data ingestion from /Users/nehiljain/code/find-your-mate-ai/data
INFO:root:OpenAI API key configured
OpenAI API key configured


(15, 17)

In [2]:
import pandas as pd
import pymongoarrow as pma
from pymongoarrow.monkey import patch_all
patch_all()

from pymongo.server_api import ServerApi

from datetime import datetime
from pymongo import MongoClient
# Create a new client and connect to the server
client = MongoClient(settings.MONGO_URI, server_api=ServerApi('1'))

database = client["db_docstore"]
collection = database["docstore/data"]
df = collection.find_pandas_all({})
# Extract metadata from the '__data__' column and expand it into separate columns in the DataFrame
metadata_df = df['__data__'].apply(lambda x: x['metadata']).apply(pd.Series)
df = df.join(metadata_df)
df = df[['_id', 'file_path', 'file_name', 'file_size', 'creation_date', 'last_modified_date']]



INFO:pymongo.serverSelection:{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 663daa977b2325b5f1fd01b9, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-qzlbp2a-shard-00-00.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-qzlbp2a-shard-00-01.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-qzlbp2a-shard-00-02.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "663daa977b2325b5f1fd01b9"}, "remainingTimeMS": 29}
{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 663daa977b2325b5f1fd01b9, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-qzlbp2a-shard-00-00.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <Server

In [3]:
df.drop_duplicates(subset=df.columns.difference(['_id']), keep='first', inplace=True)




In [4]:
db_name='db_docstore'
collection_name='docstore/data'
MONGO_URI=settings.MONGO_URI
# Fetch all existing documents from MongoDB to avoid reprocessing
existing_documents_df = fetch_all_documents_from_mongodb(MONGO_URI, db_name, collection_name)
print(existing_documents_df.shape)
existing_documents_df = existing_documents_df.head(2)
print(existing_documents_df.shape)
# Extract relevant columns for comparison
comparison_columns = ['file_path', 'file_name', 'file_size', 'creation_date', 'last_modified_date']
existing_documents_df = existing_documents_df[comparison_columns]

# Filter out documents that already exist in MongoDB
documents_df = documents_df.merge(existing_documents_df, on=comparison_columns, how='left', indicator=True)
documents_df = documents_df[documents_df['_merge'] == 'left_only'].drop(columns=['_merge'])

INFO:pymongo.serverSelection:{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 663daa9a7b2325b5f1fd01ba, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-qzlbp2a-shard-00-00.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-qzlbp2a-shard-00-01.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-qzlbp2a-shard-00-02.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "663daa9a7b2325b5f1fd01ba"}, "remainingTimeMS": 29}
{"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 663daa9a7b2325b5f1fd01ba, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-qzlbp2a-shard-00-00.dyravrx.mongodb.net', 27017) server_type: Unknown, rtt: None>, <Server

In [6]:

documents_df.file_name.values



array(['alazar-shenkute.mdx', 'albert-an.mdx', 'alex-ballasiotes.mdx',
       'alexandr-larionov.mdx', 'alireza-beig.mdx', 'alisa-amentes.mdx',
       'amit-tewari.mdx', 'andrei-ionescu.mdx', 'andrew-dopp.mdx',
       'andrew-ishak.mdx', 'andrew-sorohan.mdx', 'ankit-patel.mdx',
       'apoorv-sharma.mdx'], dtype=object)